In [208]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
import statsmodels.api as sm

from src.utils import *
from src.data_preprocessing import *
from src.data_processing import *
from src.feature_selection import *
from src.modelisation import *

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option("display.max_columns", None)


In [209]:
# pour ne pas charger le NB avec des warnings 
import warnings 
warnings.filterwarnings('ignore')

In [210]:
df = pd.read_csv('01. output/aux.csv')

In [211]:
df.drop(columns=['Unnamed: 0', 'text'], inplace=True)

In [212]:
aux = pd.melt(df, id_vars=['Speaker', 'party'])

In [213]:
aux.sort_values(by=['variable'], inplace=True)
aux = aux.reset_index(drop=True)

In [214]:
lox_de_lox = aux[['Speaker', 'party', 'value']].groupby(by=['Speaker', 'party']).sum().reset_index()
lox_de_lox.rename(columns={'value':'m_it'},inplace=True)

In [215]:
aux = pd.merge(
    aux,
    lox_de_lox,
    how='left',
    on=['Speaker', 'party']
)

In [216]:
# On Calcule les termes ds le sigma 
aux['q_hat_it'] = aux['value']/aux['m_it']

In [217]:
aux_Lab = aux.loc[aux['party'] == 'Lab']
aux_Con = aux.loc[aux['party'] == 'Con']

In [218]:
lox_de_lox = aux_Con[['variable', 'value']].groupby(by=['variable']).sum().reset_index()
lox_de_lox.rename(columns={'value':'Somme_c_it'},inplace=True)
aux_Con = pd.merge(
    aux_Con,
    lox_de_lox,
    how='left',
    on=['variable']
)
aux_Con['Somme_c_it'] = aux_Con['Somme_c_it'] - aux_Con['value']
lox_de_lox = aux.drop_duplicates(subset=['Speaker'])
lox_de_lox['Somme_m_it'] = lox_de_lox['m_it'].sum()
aux_Con = pd.merge(
    aux_Con,
    lox_de_lox[['Speaker', 'Somme_m_it']],
    how='left',
    on=['Speaker']
)
aux_Con['Somme_m_it'] = aux_Con['Somme_m_it'] - aux_Con['m_it']
aux_Con['q_hat_Con_-ijt'] = aux_Con['Somme_c_it'] / aux_Con['Somme_m_it']

In [219]:
aux_Con

,Speaker,party,variable,value,m_it,q_hat_it,Somme_c_it,Somme_m_it,q_hat_Con_-ijt
0,George Hollingbery,Con,abil,3,1948,0.001540,1918,2495823,0.000768
1,George Freeman,Con,abil,12,3886,0.003088,1909,2493885,0.000765
2,George Eustice,Con,abil,13,9312,0.001396,1908,2488459,0.000767
3,Geoffrey Cox,Con,abil,0,155,0.000000,1921,2497616,0.000769
4,Geoffrey Clifton-Brown,Con,abil,5,3940,0.001269,1916,2493831,0.000768
...,...,...,...,...,...,...,...,...,...
214995,Mary Robinson,Con,â£,10,1211,0.008258,5821,2496560,0.002332
214996,Mary Macleod,Con,â£,2,648,0.003086,5829,2497123,0.002334
214997,Martin Vickers,Con,â£,3,3692,0.000813,5828,2494079,0.002337
214998,Matt Hancock,Con,â£,40,15426,0.002593,5791,2482345,0.002333


In [220]:
lox_de_lox = aux_Lab[['variable', 'value']].groupby(by=['variable']).sum().reset_index()
lox_de_lox.rename(columns={'value':'Somme_c_it'},inplace=True)
aux_Lab = pd.merge(
    aux_Lab,
    lox_de_lox,
    how='left',
    on=['variable']
)
aux_Lab['Somme_c_it'] = aux_Lab['Somme_c_it'] - aux_Lab['value']
lox_de_lox = aux.drop_duplicates(subset=['Speaker'])
lox_de_lox['Somme_m_it'] = lox_de_lox['m_it'].sum()
aux_Lab = pd.merge(
    aux_Lab,
    lox_de_lox[['Speaker', 'Somme_m_it']],
    how='left',
    on=['Speaker']
)
aux_Lab['Somme_m_it'] = aux_Lab['Somme_m_it'] - aux_Lab['m_it']
aux_Lab['q_hat_Lab_-ijt'] = aux_Lab['Somme_c_it'] / aux_Lab['Somme_m_it']

In [221]:
aux_Lab_intial = aux_Lab.drop(columns=['q_hat_Lab_-ijt'])
aux_Lab_intial['Somme_m_it'] = aux_Lab_intial['Somme_m_it'] + aux_Lab_intial['m_it']
aux_Lab_intial['Somme_c_it'] = aux_Lab_intial['Somme_c_it'] + aux_Lab_intial['value']
aux_Lab_intial['q_hat_Lab_jt'] = aux_Lab_intial['Somme_c_it'] / aux_Lab_intial['Somme_m_it']
aux_Lab_intial = aux_Lab_intial[['variable', 'q_hat_Lab_jt']]
aux_Lab_intial.drop_duplicates(subset=['variable'], inplace=True)
aux_Lab_intial

,variable,q_hat_Lab_jt
0,abil,0.000422
440,abl,0.002813
880,absolut,0.000550
1320,accept,0.000738
1760,achiev,0.000414
...,...,...
217800,withdraw,0.000173
218240,word,0.000511
218680,write,0.000120
219120,yes,0.000463


In [222]:
# La je fais mon merge 
aux_Con = pd.merge(
    aux_Con,
    aux_Lab_intial,
    how='left',
    on=['variable']
)

In [224]:
aux_Con['rho_hat_-ijt'] = (aux_Con['q_hat_Con_-ijt']) / (aux_Con['q_hat_Con_-ijt'] + aux_Con['q_hat_Lab_jt'])

In [225]:
aux_Con

,Speaker,party,variable,value,m_it,q_hat_it,Somme_c_it,Somme_m_it,q_hat_Con_-ijt,q_hat_Lab_jt,rho_hat_-ijt
0,George Hollingbery,Con,abil,3,1948,0.001540,1918,2495823,0.000768,0.000422,0.645318
1,George Freeman,Con,abil,12,3886,0.003088,1909,2493885,0.000765,0.000422,0.644419
2,George Eustice,Con,abil,13,9312,0.001396,1908,2488459,0.000767,0.000422,0.644798
3,Geoffrey Cox,Con,abil,0,155,0.000000,1921,2497616,0.000769,0.000422,0.645512
4,Geoffrey Clifton-Brown,Con,abil,5,3940,0.001269,1916,2493831,0.000768,0.000422,0.645262
...,...,...,...,...,...,...,...,...,...,...,...
214995,Mary Robinson,Con,â£,10,1211,0.008258,5821,2496560,0.002332,0.001531,0.603579
214996,Mary Macleod,Con,â£,2,648,0.003086,5829,2497123,0.002334,0.001531,0.603853
214997,Martin Vickers,Con,â£,3,3692,0.000813,5828,2494079,0.002337,0.001531,0.604104
214998,Matt Hancock,Con,â£,40,15426,0.002593,5791,2482345,0.002333,0.001531,0.603709


In [226]:
aux_Con['produit'] = aux_Con['q_hat_it'] * aux_Con['rho_hat_-ijt']

In [229]:
nb_Con = len(aux_Con['Speaker'].unique())
nb_Con

430

In [234]:
res = aux_Con[['variable', 'produit']].groupby(by=['variable']).sum().reset_index()

In [235]:
res['moitie_Con_pi'] = res['produit']*(1/2)*(1/nb_Con)

In [236]:
res

,variable,produit,moitie_Con_pi
0,abil,0.313254,0.000364
1,abl,2.145769,0.002495
2,absolut,0.618832,0.000720
3,accept,0.411632,0.000479
4,achiev,0.334785,0.000389
...,...,...,...
495,withdraw,0.117417,0.000137
496,word,0.239652,0.000279
497,write,0.077243,0.000090
498,yes,0.260163,0.000303
